- [Linear Regression](#linear-regression)
- [Logistic Regression + MCMC + Moment Matching](#Logistic-Regression-+-MCMC-+-Moment-Matching)
- [Logistic Regression + MCMC](#Logistic-Regression-+-MCMC)
- [Logistic Regression + PP Posterior Update](#Logistic-Regression-+-PP-Posterior-Update)

In [1]:
import probpy as pp
import numpy as np
import numba
import random

linear regression
---

In [2]:
%%time
def predict(w, x):
     return x[:, 0] * w[0] + x[:, 1] * w[1] + x[:, 2] * w[2] + w[3]

w = [5, -2, 1, 5] # True underlying model
x = np.random.rand(100, 3) * 10
y = predict(w, x) + pp.normal.sample(mu=0, sigma=1, size=100).reshape(-1)



prior = pp.multivariate_normal.med(mu=np.ones(4) * 0, sigma=np.eye(4) * 5)
likelihood = pp.unilinear.med(sigma=1) # There exist an implementation for linear because it has a conjugate prior

for i in range(100):
    data = (y[i], x[i])
    
    prior = pp.parameter_posterior(data, likelihood=likelihood, priors=prior)
    
    if i % 10 == 0:
        w_approx = pp.mode(prior)
        print("Parameter Estimate", w_approx)
        
        print("Prior MSE", np.square(y - predict(w_approx, x)).mean(), 
              "True MSE", np.square(y - predict(w, x)).mean())
        print()

Parameter Estimate [0.35955699 0.87141196 0.32741238 0.10079161]
Prior MSE 487.18212204739814 True MSE 0.9664135474190029

Parameter Estimate [ 5.2373507  -1.88555814  0.99519212  3.26011582]
Prior MSE 1.2197266988880358 True MSE 0.9664135474190029

Parameter Estimate [ 5.19096202 -1.90716017  0.93536369  4.22279302]
Prior MSE 1.2075920967296943 True MSE 0.9664135474190029

Parameter Estimate [ 5.15999689 -1.92827256  1.01195044  4.00936873]
Prior MSE 1.04754122369365 True MSE 0.9664135474190029

Parameter Estimate [ 5.14230111 -2.00749384  1.06016031  4.17914481]
Prior MSE 0.9562026061592671 True MSE 0.9664135474190029

Parameter Estimate [ 5.11800727 -2.02234608  1.06406558  4.29596692]
Prior MSE 0.9263083126925267 True MSE 0.9664135474190029

Parameter Estimate [ 5.0899162  -1.99672561  1.03512442  4.31085971]
Prior MSE 0.8995829921211552 True MSE 0.9664135474190029

Parameter Estimate [ 5.09112897 -1.9979905   1.02326902  4.42608393]
Prior MSE 0.9000779386178306 True MSE 0.96641354

Logistic Regression + MCMC + Moment Matching
---

In [3]:
%%time
def sigmoid(x):
    return (1 / (1 + np.exp(-x)))

def predict(w, x):
     return x[:, 0] * w[0] + x[:, 1] * w[1] + x[:, 2] * w[2] + w[3]
    
w = [-3, 3, 5, -3] # True underlying model

x = np.random.rand(100, 3)
y = sigmoid(predict(w, x) + pp.normal.sample(mu=0.0, sigma=1.0, size=100).reshape(-1))

# For this we need custom likelihood since there is no conjugate prior

def likelihood(y, x, w):
    return pp.normal.p((y - sigmoid(x @ w[:, :-1, None] + w[:, None, None, -1]).squeeze(axis=2)),
                    mu=0.0, sigma=1.0)
    
    

prior = pp.multivariate_normal.med(mu=np.zeros(4), sigma=np.eye(4) * 5)

for i in range(50):
    j = random.randint(0, 80)
    data = (y[j: j + 20], x[j: j + 20])
    
    prior = pp.parameter_posterior(data, likelihood=likelihood, 
                                   priors=prior, 
                                   match_moments_for=pp.multivariate_normal,
                                   batch=30,
                                   samples=3000,
                                   mixing=200, 
                                   energies=1.0,
                                   mode="mcmc")

    if i % 10 == 0:
        w_approx = pp.mode(prior)
        print("Parameter Estimate", w_approx)
        
        print("Prior MSE", np.square(y - sigmoid(predict(w_approx, x))).mean(), 
              "True MSE", np.square(y - sigmoid(predict(w, x))).mean())
        print()

Parameter Estimate [-0.48975965  0.14963146  0.2350226  -1.00685625]
Prior MSE 0.11987487531336732 True MSE 0.02128228460748603

Parameter Estimate [-1.71138875  2.19177311  4.03296924 -2.69885113]
Prior MSE 0.022021863707658462 True MSE 0.02128228460748603

Parameter Estimate [-2.33872715  2.027784    4.48877893 -2.61688593]
Prior MSE 0.02133894995512444 True MSE 0.02128228460748603

Parameter Estimate [-2.7700824   2.39768265  5.14620939 -2.90258804]
Prior MSE 0.020853957575349885 True MSE 0.02128228460748603

Parameter Estimate [-3.06909024  2.80777047  5.49375224 -3.09583722]
Prior MSE 0.021046664348029345 True MSE 0.02128228460748603

CPU times: user 19.5 s, sys: 29.4 s, total: 48.9 s
Wall time: 13.1 s


Logistic Regression + MCMC
---

In [4]:
%%time
def sigmoid(x):
    return (1 / (1 + np.exp(-x)))

def predict(w, x):
     return x[:, 0] * w[0] + x[:, 1] * w[1] + x[:, 2] * w[2] + w[3]
    
w = [-3, 3, 5, -3] # True underlying model

x = np.random.rand(100, 3)
y = sigmoid(predict(w, x) + pp.normal.sample(mu=0.0, sigma=1.0, size=100).reshape(-1))

# For this we need custom likelihood since there is no conjugate prior

def likelihood(y, x, w):
    return pp.normal.p((y - sigmoid(x @ w[:, :-1, None] + w[:, None, None, -1]).squeeze(axis=2)),
                    mu=0.0, sigma=1.0)
    
    

prior = pp.multivariate_normal.med(mu=np.zeros(4), sigma=np.eye(4) * 5)

for i in range(50):
    j = random.randint(0, 80)
    data = (y[j: j + 20], x[j: j + 20])
    
    prior = pp.parameter_posterior(data, likelihood=likelihood, 
                                   priors=prior, 
                                   batch=5,
                                   samples=1000,
                                   mixing=100, 
                                   energies=1.0,
                                   mode="mcmc")

    if i % 10 == 0:
        modes = pp.mode(prior) # modes are sorted in order first is largest

        print("Number of modes", len(modes))
        w_approx = modes[0]
        
        print("Parameter Estimate", w_approx)
        
        print("Prior MSE", np.square(y - sigmoid(predict(w_approx, x))).mean(), 
              "True MSE", np.square(y - sigmoid(predict(w, x))).mean())
        print()

Number of modes 3
Parameter Estimate [-0.94819854  1.66953208  0.196584   -0.86041563]
Prior MSE 0.0687937816364164 True MSE 0.02928557812574474

Number of modes 1
Parameter Estimate [-2.20070917  1.67368058  3.28076658 -2.24555451]
Prior MSE 0.0371549385585871 True MSE 0.02928557812574474

Number of modes 1
Parameter Estimate [-2.88225696  2.7658537   4.28918435 -2.90840339]
Prior MSE 0.03134425384020127 True MSE 0.02928557812574474

Number of modes 4
Parameter Estimate [-2.94845944  3.77909141  4.72005859 -3.58584948]
Prior MSE 0.03293918569492301 True MSE 0.02928557812574474

Number of modes 5
Parameter Estimate [-2.4104538   4.67787955  5.8707972  -4.93223359]
Prior MSE 0.039656936972431625 True MSE 0.02928557812574474

CPU times: user 27.3 s, sys: 1.59 s, total: 28.9 s
Wall time: 27.4 s


Logistic Regression + PP Posterior Update
---


In [2]:
%%time
def sigmoid(x):
    return (1 / (1 + np.exp(-x)))

def predict(w, x):
     return x[:, 0] * w[0] + x[:, 1] * w[1] + x[:, 2] * w[2] + w[3]
    
w = [-3, 3, 5, -3] # True underlying model

x = np.random.rand(100, 3)
y = sigmoid(predict(w, x) + pp.normal.sample(mu=0.0, sigma=1.0, size=100).reshape(-1))

# For this we need custom likelihood since there is no conjugate prior

def likelihood(y, x, w):
    return pp.normal.p((y - sigmoid(x @ w[:, :-1, None] + w[:, None, None, -1]).squeeze(axis=2)),
                    mu=0.0, sigma=1.0)
    
    

prior = pp.multivariate_normal.med(mu=np.zeros(4), sigma=np.eye(4) * 10)

for i in range(5):
    data = (y, x)
    
    prior = pp.parameter_posterior(data, likelihood=likelihood, 
                                   priors=prior, 
                                   batch=500,
                                   samples=50000,
                                   mixing=10000, 
                                   energies=1.0,
                                   mode="ga",
                                   use_cl=True, # Set this to false to make much faster (but poorer posterior estimate)
                                   learning_rate=1.0,
                                   cl_iterations=10,
                                   verbose=True)

    modes = pp.mode(prior) # modes are sorted in order first is largest

    
    print("Number of modes", len(modes))
    for mode in modes:
        print(mode)

    w_approx = modes[0]

    print("Parameter Estimate", w_approx)

    print("Prior MSE", np.square(y - sigmoid(predict(w_approx, x))).mean(), 
          "True MSE", np.square(y - sigmoid(predict(w, x))).mean())
    print()

error: 2.5119886317496816
error: 2.400955434781392
error: 2.33858510613364
error: 2.298351443616884
error: 2.269222083340171
error: 2.2462993855133946
error: 2.2272378140668088
error: 2.211264398499265
error: 2.1975485681821585
error: 2.18612205382595
Number of modes 5
[-1.17199987  2.91957555  2.56055134 -2.61223954]
[-1.81207922  1.5198941   1.82838972 -0.92995165]
[-1.72604362  0.60443303  2.67814707 -1.44786454]
[-2.83478365  2.27942386  3.20341116 -1.77437739]
[ 1.39674632  1.54538525  1.90670414 -2.93755185]
Parameter Estimate [-1.17199987  2.91957555  2.56055134 -2.61223954]
Prior MSE 0.02625467092230763 True MSE 0.02883993811449654

error: 1.5826776770538862
error: 1.5078969783384788
error: 1.4486842738345742
error: 1.4028603273588487
error: 1.3668298614174805
error: 1.3381779889917438
error: 1.3161743322041053
error: 1.2988155436457776
error: 1.2848226072332072
error: 1.2735684074485984
Number of modes 2
[-1.36480088  2.92253043  2.34947362 -2.42660247]
[-2.08950591  1.7539419

IRT 
---

In [ ]:

# TODO